In [1]:
import torchsummary
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
# from model_utils import EncodingLayer, SphericalMaxPooling

In [2]:
SF1 = np.random.randint(0, 3, (642, 3)), np.random.randint(-1, 642, (642, 6))
SF2 = np.random.randint(0, 3, (162, 3)), np.random.randint(-1, 162, (162, 6))
SF3 = np.random.randint(0, 3, (42, 3)), np.random.randint(-1, 42, (42, 6))
SFs = [SF1, SF2, SF3]

In [3]:
def readIcosahedron(file_path, n_vertices):       #정리해놓은 Icosahedron 파일 읽어서 SF return
    vertices = np.zeros((n_vertices, 3))
    near_idx = np.zeros((n_vertices, 6)) - 1
    faces = np.zeros(((n_vertices-2)*2, 3))

    cnt_v = 0
    cnt_n = 0
    cnt_f = 0   

    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            temp = line.split()

            if temp[0] == 'v':
                vertices[cnt_v, 0] = temp[1]
                vertices[cnt_v, 1] = temp[2]
                vertices[cnt_v, 2] = temp[3]
                cnt_v += 1

            if temp[0] == 'f':
                faces[cnt_f, 0] = temp[1]
                faces[cnt_f, 1] = temp[2]
                faces[cnt_f, 2] = temp[3]
                cnt_f += 1

            if temp[0] == 'np':
                near_idx[cnt_n, 0] = temp[1]
                near_idx[cnt_n, 1] = temp[2]
                near_idx[cnt_n, 2] = temp[3]
                near_idx[cnt_n, 3] = temp[4]
                near_idx[cnt_n, 4] = temp[5]
                if len(temp)>=7:
                    near_idx[cnt_n, 5] = temp[6]

                cnt_n += 1

    return vertices, near_idx, faces

SF1 = readIcosahedron('./data_utils/SF1.txt', 42)
SF2 = readIcosahedron('./data_utils/SF2.txt', 162)
SF3 = readIcosahedron('./data_utils/SF3.txt', 642)

In [4]:
SFs = [SF1, SF2, SF3]
v, n, f = SF1
v.shape, f.shape

((42, 3), (80, 3))

In [5]:
from models.SFCNN_seg import get_model

In [9]:
xyz = torch.rand((8, 3, 2048)).cuda()
projected = torch.rand(8, 1, 642).cuda()
cls_label = torch.randint(16, size=(8, 16)).cuda()

In [10]:
model = get_model(16, SFs).cuda()

In [11]:
model(xyz, projected, cls_label)

torch.Size([8, 64, 642])
torch.Size([8, 128, 642])
torch.Size([8, 256, 162])
torch.Size([8, 1024, 42])


RuntimeError: All input tensors must be on the same device. Received cpu and cuda:0

In [4]:
torchsummary.summary(get_model(16, SFs).cuda(), input_size=(1, 642), batch_size=24)

torch.Size([2, 16]) torch.Size([2, 16, 642])
torch.Size([2, 32]) torch.Size([2, 32, 162])
torch.Size([2, 64]) torch.Size([2, 64, 42])
torch.Size([2, 128]) torch.Size([2, 128, 42])
torch.Size([2, 480])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [24, 16, 6, 642]              48
       BatchNorm2d-2           [24, 16, 6, 642]              32
            Conv2d-3           [24, 16, 6, 642]             272
       BatchNorm2d-4           [24, 16, 6, 642]              32


/home/rmclab102/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


TypeError: can't multiply sequence by non-int of type 'list'

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from model_utils import EncodingLayer, SphericalMaxPooling

def makeInputData(input_feature, SF):
    #feature에서 근점 점 6개 인덱스 뽑아서 MLP 입력 블럭으로 만들어놔
    #output : B x N x 6 x 2C (기준 점x6 로 늘려서 근접 점이랑 붙이삼)
    '''
    input : B x N x C
    SF : N x (xyz + p1~p6)

    input*6 + p1~p6 (concat) => output

    output : B x N x 6 x 2C
    '''
    #임시로 해놓음
    # input_feature = input_feature.numpy()
    input_feature = torch.Tensor(input_feature)
    # print(input_feature.shape)
    B, N, C = input_feature.shape
    vertices, near_idx = SF         #near_idx : Nx6
    # features = np.reshape(input_feature, (B, N, 1, C))
    # input_feature = input_feature.permute(0, 2, 1)
    features = input_feature.view(B, N, 1, C)
    # features = np.repeat(features, 6, axis=2)   #BxNx6xC
    features = features.repeat(1, 1, 6, 1)    #BxNx6xC

    # outofidx_6 = np.zeros((B, N, C)) - 1
    outofidx_6 = torch.zeros((B, N, C)) - 1
    # input_feature = np.concatenate((input_feature, outofidx_6), axis=1) #Bx(N+1)xC

    input_feature =  torch.cat((input_feature, outofidx_6), 1)    #Bx(N+1)xC

    near_features = input_feature[:, near_idx, :]   #BxNx6xC

    # feature_block = np.concatenate((features, near_features), axis=-1)  #BxNx6x2C
    feature_block = torch.cat((features, near_features), 3)  #BxNx6x2C

    # feature_block = torch.Tensor(feature_block)

    return feature_block    #BxNx6x2C

def SphericalMaxPooling(input, in_SF, out_SF):
    '''
    input : B x C x N0
    SF0 : (xyz, p01~p05 idx) - (N0 x 3), (N0 x 6)
    SF1 : (xyz, p10~p15 idx) - (N1 x 3), (N1 x 6)

    output : B x N1 x C

    for SF1:
        for SF2:
            SF2-xyz중에 SF1이랑 같은거 찾아서 p01~p06 인덱스 input에서 d 꺼내와서 pooling
            그 값 ouput에 추가
    '''
    B, C, N = input.shape
    input = input.permute(0, 2, 1)

    in_v, in_ni = in_SF
    out_v, out_ni = out_SF

    in_nvertices = in_v.shape[0]
    out_nvertices = out_v.shape[0]

    output = torch.zeros((B, out_nvertices, C))
    
    for i in range(out_nvertices):
        out_xyz = out_v[i]
        for j in range(in_nvertices):
            if out_xyz.all() == in_v[j].all():
                near_idx = in_ni[j]  #Bx6
                temp = input[:, near_idx, :]   #Bx6xC
                output[:, i, :] = torch.max(temp, 1)[0]     #BxC

    input = input.permute(0, 2, 1)

    return output   #BxCxN'

    # # input : B x N x (xyz + d + p1~p6 + alpha)로 먼저 만드삼
    # # alpha - pooling 기준 점인지 아닌지 (0 : x, 1 : o)
    # # for i in range(N):
    # #   a=0이면 pass
    # #   a=1이면 p1~p6 d값 포함 maxpooling 하고 p1~p6 a값 0으로 변경
    # # 그리고 a=0인 애들 다 죽이삼
    # # output : B x N x (xyz + d)
    # 이거 아님

class SphericalMaxPooling(nn.Module):
    def __init__(self, in_SF, out_SF):
        super(SphericalMaxPooling, self).__init__()
        self.in_SF = in_SF
        self.out_SF = out_SF
    '''
    input : B x C x N0
    SF0 : (xyz, p01~p05 idx) - (N0 x 3), (N0 x 6)
    SF1 : (xyz, p10~p15 idx) - (N1 x 3), (N1 x 6)

    output : B x N1 x C

    for SF1:
        for SF2:
            SF2-xyz중에 SF1이랑 같은거 찾아서 p01~p06 인덱스 input에서 d 꺼내와서 pooling
            그 값 ouput에 추가
    '''

    def forward(self, input):
        B, C, N = input.shape
        input = input.permute(0, 2, 1)

        in_v, in_ni = self.in_SF
        out_v, out_ni = self.out_SF

        in_nvertices = in_v.shape[0]
        out_nvertices = out_v.shape[0]

        output = torch.zeros((B, out_nvertices, C))
        
        for i in range(out_nvertices):
            out_xyz = out_v[i]
            for j in range(in_nvertices):
                if out_xyz.all() == in_v[j].all():
                    near_idx = in_ni[j]  #Bx6
                    temp = input[:, near_idx, :]   #Bx6xC
                    output[:, i, :] = torch.max(temp, 1)[0]     #BxC

        output = output.permute(0, 2, 1)

        return output   #BxCxN'



In [4]:
import torch.nn as nn
import torch.nn.functional as F
# from models.SFCNN_utils import EncodingLayer, SphericalMaxPooling
# from models.SFCNN_utils import EncodingLayer


class get_model(nn.Module):
    def __init__(self, num_class, SF_list, normal_channel=False):
        super(get_model, self).__init__()
        in_channel = 3 if normal_channel else 1
        self.normal_channel = normal_channel
        self.SF1, self.SF2, self.SF3 = SF_list[2], SF_list[1], SF_list[0]

        self.layer0 = EncodingLayer(self.SF3, in_channel, [16, 16]) 
        self.layer1 = EncodingLayer(self.SF3, 16, [32, 32])   
        self.layer2 = EncodingLayer(self.SF2, 32, [64, 64])   
        self.layer3 = EncodingLayer(self.SF1, 64, [128, 128])   

        self.smp1 = SphericalMaxPooling(self.SF3, self.SF2)
        self.smp2 = SphericalMaxPooling(self.SF2, self.SF1)

        self.fc1 = nn.Linear(480, 512)
        self.bn1 = nn.BatchNorm1d(512)
        # self.drop1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(512, 128)
        self.bn2 = nn.BatchNorm1d(128)
        # self.drop2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, num_class)

    def forward(self, xyz):
        B, _, _ = xyz.shape      #Bx1x642
        # if self.normal_channel:
        #     norm = xyz[:, 3:, :]
        #     xyz = xyz[:, :3, :]
        # else:
        #     norm = None
        # print(xyz.shape)

        

        l0_inner, l0_outer, l0_feature = self.layer0(xyz)       #Bx16x642
        # SphericalMaxPooling(l0_feature, self.SF3, self.SF3)
        print(l0_inner.shape, l0_feature.shape)

        l1_inner, l1_outer, l1_feature = self.layer1(l0_feature)    #Bx32x642
        l1_out = self.smp1(l1_feature)     #Bx32x162
        print(l1_inner.shape, l1_out.shape)
        
        l2_inner, l2_outer, l2_feature = self.layer2(l1_out)     #Bx64x162
        l2_out = self.smp2(l2_feature)     #Bx64x42
        print(l2_inner.shape, l2_out.shape)
        
        l3_inner, l3_outer, l3_feature = self.layer3(l2_out)     #Bx128x42
        print(l3_inner.shape, l3_feature.shape)

        # l0_inner = torch.max(l0_inner, 2)[0]
        # l0_feature = torch.max(l0_feature, 2)[0]
        # l1_inner = torch.max(l1_inner, 2)[0]
        # l1_feature = torch.max(l1_feature, 2)[0]
        # l2_inner = torch.max(l2_inner, 2)[0]
        # l2_feature = torch.max(l2_feature, 2)[0]
        # l3_inner = torch.max(l3_inner, 2)[0]
        # l3_feature = torch.max(l3_feature, 2)[0]

        out_feature = torch.cat((l0_inner, l0_outer, l1_inner, l1_outer, l2_inner, l2_outer, l3_inner, l3_outer), 1)

        print(out_feature.shape)
        x = out_feature.view(B, -1)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)
        x = F.log_softmax(x, -1)

        return x, l3_feature


class get_loss(nn.Module):
    def __init__(self):
        super(get_loss, self).__init__()

    def forward(self, pred, target):
        total_loss = F.nll_loss(pred, target)

        return total_loss

In [2]:
import torchsummary
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
# from model_utils import EncodingLayer, SphericalMaxPooling
SF3 = np.random.randint(0, 3, (642, 3)), np.random.randint(-1, 642, (642, 6))
SF2 = np.random.randint(0, 3, (162, 3)), np.random.randint(-1, 162, (162, 6))
SF1 = np.random.randint(0, 3, (42, 3)), np.random.randint(-1, 42, (42, 6))
SFs = [SF3, SF2, SF1]

In [3]:
class EncodingLayer(nn.Module):
    def __init__(self, SF, in_channel, mlp, residual=False):
        super(EncodingLayer, self).__init__()
        self.SF = SF
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel*2

        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))  #왜 2d로 함?
            self.mlp_bns.append(nn.BatchNorm2d(out_channel))
            last_channel = out_channel

    def forward(self, points):
        """
        Input:
            points: input points data, [B, C, N]
        Return:
            new_xyz: sampled points position data, [B, C', N]
        """
        points = points.permute(0, 2, 1)    #BxNxC

        feature_block = makeInputData(points, self.SF)      #BxNx6x2C
        new_feature = feature_block.permute(0, 3, 2, 1)    #Bx2Cx6xN

        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_feature =  F.relu(bn(conv(new_feature)))

            if i==0:
                inner_feature = new_feature
        
        new_feature = torch.max(new_feature, 2)[0] #[B, C', N] - maxpooling of conv (1 kernel)
        inner_feature = torch.max(inner_feature, 2)[0]

        inner_feature = torch.max(inner_feature, 2)[0]
        out_feature = torch.max(new_feature, 2)[0]
        # print("ff")
#         new_feature = new_feature.permute(0, 2, 1)
#         print(new_feature.shape)

        return inner_feature, out_feature, new_feature

In [56]:
model = get_model(20, SFs)

In [57]:
model(a)

torch.Size([20, 16]) torch.Size([20, 16, 642])
torch.Size([20, 32]) torch.Size([20, 32, 162])
torch.Size([20, 64]) torch.Size([20, 64, 42])
torch.Size([20, 128]) torch.Size([20, 128, 42])
torch.Size([20, 480])


(tensor([[-2.9822, -3.0191, -2.9724, -2.9181, -3.0396, -3.0406, -2.9642, -3.0403,
          -3.0055, -3.0040, -2.9219, -3.0157, -3.0636, -3.0404, -2.9077, -3.0463,
          -2.9752, -3.0383, -2.9233, -3.0189],
         [-2.9822, -3.0191, -2.9724, -2.9181, -3.0396, -3.0406, -2.9642, -3.0403,
          -3.0055, -3.0040, -2.9219, -3.0157, -3.0636, -3.0404, -2.9077, -3.0463,
          -2.9752, -3.0383, -2.9233, -3.0189],
         [-2.9822, -3.0191, -2.9724, -2.9181, -3.0396, -3.0406, -2.9642, -3.0403,
          -3.0055, -3.0040, -2.9219, -3.0157, -3.0636, -3.0404, -2.9077, -3.0463,
          -2.9752, -3.0383, -2.9233, -3.0189],
         [-2.9822, -3.0191, -2.9724, -2.9181, -3.0396, -3.0406, -2.9642, -3.0403,
          -3.0055, -3.0040, -2.9219, -3.0157, -3.0636, -3.0404, -2.9077, -3.0463,
          -2.9752, -3.0383, -2.9233, -3.0189],
         [-2.9822, -3.0191, -2.9724, -2.9181, -3.0396, -3.0406, -2.9642, -3.0403,
          -3.0055, -3.0040, -2.9219, -3.0157, -3.0636, -3.0404, -2.9077, -

In [27]:
a = torch.zeros(20, 1, 642)

In [6]:

def readIcosahedron(file_path, n_vertices):       #정리해놓은 Icosahedron 파일 읽어서 SF return
    vertices = np.zeros((n_vertices, 3))
    near_idx = np.zeros((n_vertices, 6)) - 1

    cnt_v = 0
    cnt_n = 0

    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            temp = line.split()

            if temp[0] == 'v':
                vertices[cnt_v, 0] = temp[1]
                vertices[cnt_v, 1] = temp[2]
                vertices[cnt_v, 2] = temp[3]
                cnt_v += 1

            if temp[0] == 'np':
                near_idx[cnt_n, 0] = temp[1]
                near_idx[cnt_n, 1] = temp[2]
                near_idx[cnt_n, 2] = temp[3]
                near_idx[cnt_n, 3] = temp[4]
                near_idx[cnt_n, 4] = temp[5]
                if len(temp)>=7:
                    near_idx[cnt_n, 5] = temp[6]

                cnt_n += 1

    return vertices, near_idx

In [7]:
SF1 = readIcosahedron('./data_utils/SF1.txt', 42)
SF2 = readIcosahedron('./data_utils/SF2.txt', 162)
SF3 = readIcosahedron('./data_utils/SF3.txt', 642)

In [20]:
in_v, in_ni = SF2
out_v, out_ni = SF1

in_nvertices = in_v.shape[0]
out_nvertices = out_v.shape[0]

In [21]:
in_nvertices

162

In [22]:
with open('./data_utils/SF3toSF2_idxs.txt', 'w') as f:
    for i, out_xyz in enumerate(out_v):
        for j, in_xyz in enumerate(in_v):
            if out_xyz[0] == in_xyz[0] and out_xyz[1] == in_xyz[1] and out_xyz[2] == in_xyz[2]:
#                 f.write('%d\n' % j)
                print(i, j)
                break

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41


In [ ]:
with open(out_path, 'w') as f:
                for v in vertices:
                    f.write('v')
                    for p in v:
                        f.write(' %f' % p)
                    f.write("\n")

In [7]:
def cvtIcosahedron(in_path, out_path):      #Icosahedron .obj 읽어서 nearest point 6개 찾아서 파일로 저장
    textured_mesh = o3d.io.read_triangle_mesh(in_path)
    vertices = np.asarray(textured_mesh.vertices)
    triangle = np.asarray(textured_mesh.triangles)

    edge = [[] for i in range(len(vertices))]

    for t in triangle:
        for idx in t:
            for i in t:
                if idx!=i and i not in edge[idx]:
                    edge[idx].append(i)

    with open(out_path, 'w') as f:
        for v in vertices:
            f.write('v')
            for p in v:
                f.write(' %f' % p)
            f.write("\n")

        for t in triangle:
            f.write('f')
            for idx in t:
                f.write(' %d' % idx)
            f.write("\n")

        for nnp in edge:
            f.write('np')
            for p in nnp:
                f.write(' %d' % int(p))
            f.write("\n")